In [ ]:
import os, sys, shutil, subprocess, matplotlib
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import spatial
path   = os.getcwd()
pathrf = path+'/opt_refframe'
pathpdb  = path+'/opt_pdb'

In [ ]:
def dna_data(refframe_file, pdb_file):
    """
    Function that will read both a reference frame file and a file of phosphate atoms from a pdb file.
    Goal will be to return a pandas DataFrame with information such as:
    - base-pair origin
    - base-pair's direction of its minor groove
    - the phosphate positions on both its coding (Pi) and complementary (pi) strands
    """
    infile1   = open(refframe_file, 'r')
    infile2   = open(pdb_file, 'r')
    rfdata    = infile1.readlines()
    PDBdata   = infile2.readlines()
    infile1.close()
    infile2.close()
    
    df = pd.DataFrame(columns=["Ox","Oy","Oz",
                               "Tox","Toy","Toz",
                               "-d1x","-d1y","-d1z",
                               "px","py","pz",
                               "qx","qy","qz"])    
    Ncirc = int(rfdata[0].split()[0])
    rfdata  = [i.split() for i in rfdata]
    Odata = []
    Xdata = []
    for i in range(0, Ncirc):
        Odata.append(rfdata[(5*i)+2])
        Xdata.append([rfdata[(5*i)+3][0],
                      rfdata[(5*i)+4][0],
                      rfdata[(5*i)+5][0]])
    for i in range(0, len(Odata)):
        df.loc[i, ["Ox","Oy","Oz"]]     = float(Odata[i][0]), float(Odata[i][1]), float(Odata[i][2])
        df.loc[i, ["-d1x","-d1y","-d1z"]] = -1*float(Xdata[i][0]), -1*float(Xdata[i][1]), -1*float(Xdata[i][2])
    
    com = np.array([float((df['Ox'].sum())/Ncirc),
                    float((df['Oy'].sum())/Ncirc),
                    float((df['Oz'].sum())/Ncirc)])
    for i in range(0, len(Odata)):
        df.loc[i, ["Tox","Toy","Toz"]] = (np.array(df.loc[i, ["Ox","Oy","Oz"]]) - com)
    
    Pdata = []
    for i in range(0, len(PDBdata)):
        if " P " in PDBdata[i]:
            Pdata.append(PDBdata[i])
    Pdata = [[i[30:38],i[38:46],i[46:54]] for i in Pdata]
    # if this opt as a circle, 1 and Ncirc+1 will be identical    
    if len(Pdata) == 2*Ncirc + 2:
        for i in range(0, len(Odata)):
            df.loc[i, ["px","py","pz"]] = float(Pdata[i][0]), float(Pdata[i][1]), float(Pdata[i][2])
            j = ((2*Ncirc + 2)-i)-1
            df.loc[i, ["qx","qy","qz"]] = float(Pdata[j][0]), float(Pdata[j][1]), float(Pdata[j][2])
    #del df
    del rfdata, PDBdata, Odata, Xdata, Pdata
    df = minor_groove(df)
    df = major_groove(df)

    return df

def minor_groove(dataframe):
    """
    To calculate the width of the minor groove for a circular structure.
    For step 'k', get the average of distances between a pair of phosphates offset by m=-3.
    A: distance from coding p(k+1) to complementary q(k-2)
    B: distance from coding p(k+2) to complementary q(k-1)
    *** For this function, the p phosphates will be complementary to the k-1th step's bp number
    *** for k=10 and circular N=150, p((k+1)+2)=p(k+3)=p([13])
    """
    Nseq = len(dataframe)
    for k in range(0, Nseq):
        if k == 0:
            Pk1 = np.array([dataframe.loc[2,      ["px","py","pz"]]])
            Pk2 = np.array([dataframe.loc[3,      ["px","py","pz"]]])
            pk1 = np.array([dataframe.loc[Nseq-1, ["qx","qy","qz"]]])
            pk2 = np.array([dataframe.loc[Nseq-2, ["qx","qy","qz"]]])
        elif k == 1:
            Pk1 = np.array([dataframe.loc[3,      ["px","py","pz"]]])
            Pk2 = np.array([dataframe.loc[4,      ["px","py","pz"]]])
            pk1 = np.array([dataframe.loc[0,      ["qx","qy","qz"]]])
            pk2 = np.array([dataframe.loc[Nseq-1, ["qx","qy","qz"]]])
        elif k == Nseq-3:
            Pk1 = np.array([dataframe.loc[k+2,    ["px","py","pz"]]])
            Pk2 = np.array([dataframe.loc[0,      ["px","py","pz"]]])
            pk1 = np.array([dataframe.loc[k-1,    ["qx","qy","qz"]]])
            pk2 = np.array([dataframe.loc[k-2,    ["qx","qy","qz"]]])
        elif k == Nseq-2:
            Pk1 = np.array([dataframe.loc[0,      ["px","py","pz"]]])
            Pk2 = np.array([dataframe.loc[1,      ["px","py","pz"]]])
            pk1 = np.array([dataframe.loc[k-1,    ["qx","qy","qz"]]])
            pk2 = np.array([dataframe.loc[k-2,    ["qx","qy","qz"]]])
        elif k == Nseq-1:
            Pk1 = np.array([dataframe.loc[1,      ["px","py","pz"]]])
            Pk2 = np.array([dataframe.loc[2,      ["px","py","pz"]]])
            pk1 = np.array([dataframe.loc[k-1,    ["qx","qy","qz"]]])
            pk2 = np.array([dataframe.loc[k-2,    ["qx","qy","qz"]]])
        else:
            Pk1 = np.array([dataframe.loc[k+2,    ["px","py","pz"]]])
            Pk2 = np.array([dataframe.loc[k+3,    ["px","py","pz"]]])
            pk1 = np.array([dataframe.loc[k-1,    ["qx","qy","qz"]]])
            pk2 = np.array([dataframe.loc[k-2,    ["qx","qy","qz"]]])
        
        A = sp.spatial.distance.euclidean(Pk1, pk2)
        B = sp.spatial.distance.euclidean(Pk2, pk1)
        mg = (1/2)*( A + B )
        dataframe.loc[k, "W-min"] = mg
    del Nseq
    return dataframe

def major_groove(dataframe):
    """
    To calculate the width of the major groove for a circular structure.
    For step 'k', get the average of distances between a pair of phosphates offset by m=4.
    mg = distance from coding p((k+1)-2)=p(k-1) to complementary q(k+2)
    """
    Nseq = len(dataframe)
    for k in range(0, Nseq):
        if k == 0:
            Pk2 = np.array([dataframe.loc[Nseq-1, ["px","py","pz"]]])
            pk2 = np.array([dataframe.loc[2,      ["qx","qy","qz"]]])
        elif k == 1:
            Pk2 = np.array([dataframe.loc[0,      ["px","py","pz"]]])
            pk2 = np.array([dataframe.loc[3,      ["qx","qy","qz"]]])
        elif k == Nseq-2:
            Pk2 = np.array([dataframe.loc[k-1,    ["px","py","pz"]]])
            pk2 = np.array([dataframe.loc[0,      ["qx","qy","qz"]]])
        elif k == Nseq-1:
            Pk2 = np.array([dataframe.loc[k-1,    ["px","py","pz"]]])
            pk2 = np.array([dataframe.loc[1,      ["qx","qy","qz"]]])
        else:
            Pk2 = np.array([dataframe.loc[k-1,    ["px","py","pz"]]])
            pk2 = np.array([dataframe.loc[k+2,    ["qx","qy","qz"]]])
            
        mg = sp.spatial.distance.euclidean(Pk2, pk2)
        dataframe.loc[k, "W-maj"] = mg
    del Nseq
    return dataframe


In [ ]:
lst = []
for filename in os.listdir(pathrf):
    lst.append(filename.split('.')[0])
lst = sorted(lst)
print(lst[0])
for l in range(0, 1):
    df  = dna_data(pathrf +'/'+lst[l]+'.dat', pathpdb+'/'+lst[l]+'.pdb')


In [ ]:
for l in range(0, len(lst)):
    if not os.path.exists(path+'/opt_grooves/'+lst[l]):
        df  = dna_data(pathrf +'/'+lst[l]+'.dat', pathpdb+'/'+lst[l]+'.pdb')
        df.to_csv(path+'/opt_grooves/'+lst[l])
        del df

In [ ]:
# --- Compare with groove information in a .out file

infile = open(path+'/opt_out/'+lst[0]+'.out', 'r')
indata = infile.readlines()
indata = indata[1776:1925]
indata = [i.replace('---', '0.0') for i in indata]
indata = [i.split() for i in indata]
mgdf = pd.DataFrame.from_records(indata)
mgdf = mgdf.drop(columns=[0], axis=1)
del indata
infile.close()
mgdf[2] = mgdf[2].astype("float64")
mgdf[3] = mgdf[3].astype("float64")
mgdf[4] = mgdf[4].astype("float64")
mgdf[5] = mgdf[5].astype("float64")